In [19]:
import numpy as np
import pandas as pd
from PIL import Image

import torchvision
import torchvision.transforms as transforms

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as func
import torch.optim as optim

In [20]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [21]:
from datasets import load_dataset

ds = load_dataset("uoft-cs/cifar100")

In [22]:
class CustomCIFAR100Dataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.hf_dataset = hf_dataset
        self.transform = transform
    
    def __len__(self):
        return len(self.hf_dataset)
    
    def __getitem__(self, idx):
        # Get the image and label from the Hugging Face dataset
        img_data = self.hf_dataset[idx]['img']
        label = self.hf_dataset[idx]['fine_label']
        
        # Apply transformations
        if self.transform:
            img = self.transform(img_data)
        
        return img, label

In [23]:
train_dataset = CustomCIFAR100Dataset(hf_dataset=ds['train'], transform=transform)
test_dataset = CustomCIFAR100Dataset(hf_dataset=ds['test'], transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [24]:
img, label = train_dataset[0]



In [25]:
img.size()

torch.Size([3, 32, 32])

In [26]:
# class_names = ['dog', 'horse', 'deer', 'car', 'bird']
classes_totrain = [
    "apple", "aquarium_fish", "baby", "bear", "beetle", "bicycle", "bottle", "bowl",
    "boy", "bridge", "bus", "butterfly", "camel", "can", "castle", "caterpillar",
    "chair", "chicken", "cloud", "cockroach", "couch", "crab", "cup", "dolphin",
    "elephant", "flatfish", "forest", "fox", "girl", "hamster", "house", "kangaroo",
    "keyboard", "lamp", "lawn_mower", "leopard", "lion", "lizard", "man", "maple_tree",
    "motorcycle", "mountain", "mouse", "mushroom", "oak_tree", "orange", "orchid",
    "otter", "palm_tree", "pear", "pickup_truck", "pine_tree", "plain", "plate",
    "poppy", "porcupine", "possum", "rabbit", "raccoon", "ray", "road", "rocket",
    "rose", "sea", "seal", "shark", "sheep", "skunk", "skyscraper", "snowboard",
    "soccer_ball", "spider", "squirrel", "streetcar", "sunflower", "sweet_pepper",
    "table", "tank", "telephone", "television", "tiger", "tractor", "train", "truck",
    "umbrella", "vase", "wheat", "wheelchair", "zebra", "airplane", "automobile",
    "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck", "train"
]

In [27]:
class ImprovedNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 100)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(func.relu(self.bn1(self.conv1(x))))
        x = self.pool(func.relu(self.bn2(self.conv2(x))))
        x = self.pool(func.relu(self.bn3(self.conv3(x))))
        x = torch.flatten(x, 1)
        x = func.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [28]:
net = ImprovedNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)

In [29]:
for epoch in range(60):
    print(f'Training epoch {epoch}...')
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Loss: {running_loss/len(train_loader):.4f}')
    scheduler.step(running_loss)

Training epoch 0...
Loss: 4.3056
Training epoch 1...
Loss: 4.0790
Training epoch 2...
Loss: 3.9597
Training epoch 3...
Loss: 3.8414
Training epoch 4...
Loss: 3.7581
Training epoch 5...
Loss: 3.6753
Training epoch 6...
Loss: 3.5841
Training epoch 7...
Loss: 3.4970
Training epoch 8...
Loss: 3.4007
Training epoch 9...
Loss: 3.3197
Training epoch 10...
Loss: 3.2290
Training epoch 11...
Loss: 3.1346
Training epoch 12...
Loss: 3.0570
Training epoch 13...
Loss: 2.9469
Training epoch 14...
Loss: 2.8412
Training epoch 15...
Loss: 2.7677
Training epoch 16...
Loss: 2.6953
Training epoch 17...
Loss: 2.6261
Training epoch 18...
Loss: 2.5484
Training epoch 19...
Loss: 2.4982
Training epoch 20...
Loss: 2.4347
Training epoch 21...
Loss: 2.3818
Training epoch 22...
Loss: 2.3357
Training epoch 23...
Loss: 2.2907
Training epoch 24...
Loss: 2.2329
Training epoch 25...


In [12]:
torch.save(net.state_dict(), 'trained_network.pth')

In [13]:
net = ImprovedNet()
net.load_state_dict(torch.load('trained_network.pth'))

C:\Users\hp\AppData\Local\Temp\ipykernel_11060\1038664596.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_network.pth'))


<All keys matched successfully>

In [14]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 51.69%


In [1]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

# Testing with new images
image_paths = ['lion.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {classes_totrain[predicted.item()]}')

NameError: name 'transforms' is not defined